###############################################################################
# SRSWTISentimentAnalyzer
###############################################################################


In [ ]:
# ---------------------------  Imports ------------------------------
from srswti_axis import SRSWTISentimentAnalyzer
from utils import *

/home/zhreyas/miniconda3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-10 20:19:52.524146: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741618192.546250 1103118 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741618192.551967 1103118 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-10 20:19:52.574492: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-c

# ---------------------------  Basic Demos ------------------------


In [ ]:
# Initialize the sentiment analyzer
sent_analyzer = SRSWTISentimentAnalyzer()

In [5]:
txt_s1 = "I absolutely love the new camera's features!"
print("== Sentiment 1 ==")
print(sent_analyzer.analyze(txt_s1), "\n")


== Sentiment 1 ==
{'overall': {'sentiment': {'compound': 0.6989, 'positive': 0.489, 'negative': 0.0, 'neutral': 0.511}, 'text_stats': {'sentence_count': 1, 'word_count': 9, 'avg_sentence_length': 9.0}}, 'sentences': [{'text': "I absolutely love the new camera's features!", 'sentiment': SRSWTISentiment(compound=0.6989, positive=0.489, negative=0.0, neutral=0.511), 'intensifiers': [{'intensifier': 'absolutely', 'modified_word': 'love'}], 'length': 9}], 'aspects': {}, 'summary': 'The overall sentiment is very positive'} 



In [ ]:
# Define a sample text about a restaurant experience
txt_s2 = "The restaurant was expensive but the service was friendly and the food was delicious."

# Print a heading for this sentiment analysis example
print("== Sentiment 2 ==")

# Analyze the sentiment of the text with specific aspects and domain
# - aspects=["food","service"]: Focus analysis on these specific aspects
# - domain="restaurant": Provide context that we're analyzing restaurant reviews
# This will return sentiment scores for the overall text and the specified aspects
print(sent_analyzer.analyze(txt_s2, aspects=["food","service"], domain="restaurant"), "\n")

== Sentiment 2 ==
{'overall': {'sentiment': {'compound': 0.8847, 'positive': 0.438, 'negative': 0.0, 'neutral': 0.562}, 'text_stats': {'sentence_count': 1, 'word_count': 15, 'avg_sentence_length': 15.0}}, 'sentences': [{'text': 'The restaurant was expensive but the service was friendly and the food was delicious.', 'sentiment': SRSWTISentiment(compound=0.8847, positive=0.438, negative=0.0, neutral=0.562), 'intensifiers': [], 'length': 15}], 'aspects': {'food': {'mentions': [{'text': 'food', 'sentiment_score': 0.8847, 'context': 'The restaurant was expensive but the service was friendly and the food was delicious.', 'position': (8, 15), 'modifiers': ['friendly', 'delicious']}]}, 'service': {'mentions': [{'text': 'service', 'sentiment_score': 0.8847, 'context': 'The restaurant was expensive but the service was friendly and the food was delicious.', 'position': (3, 10), 'modifiers': ['expensive', 'friendly']}]}}, 'summary': 'The overall sentiment is very positive. food is mentioned 1 time

In [7]:
txt_s3 = "The laptop battery is terrible, but the CPU speed is amazing."
print("== Sentiment 3 ==")
print(sent_analyzer.analyze(txt_s3, aspects=["battery","CPU"], domain="product"), "\n")

== Sentiment 3 ==
{'overall': {'sentiment': {'compound': 0.631, 'positive': 0.32, 'negative': 0.126, 'neutral': 0.554}, 'text_stats': {'sentence_count': 1, 'word_count': 13, 'avg_sentence_length': 13.0}}, 'sentences': [{'text': 'The laptop battery is terrible, but the CPU speed is amazing.', 'sentiment': SRSWTISentiment(compound=0.631, positive=0.32, negative=0.126, neutral=0.554), 'intensifiers': [], 'length': 13}], 'aspects': {'battery': {'mentions': [{'text': 'battery', 'sentiment_score': 0.631, 'context': 'The laptop battery is terrible, but the CPU speed is amazing.', 'position': (0, 6), 'modifiers': ['laptop', 'terrible']}]}, 'CPU': {'mentions': [{'text': 'CPU', 'sentiment_score': 0.631, 'context': 'The laptop battery is terrible, but the CPU speed is amazing.', 'position': (5, 12), 'modifiers': []}]}}, 'summary': 'The overall sentiment is very positive. battery is mentioned 1 times and is generally positive. CPU is mentioned 1 times and is generally positive'} 



# ------------------- RAG Demos ---------------------


In [ ]:
def rag_demo_a(docs=SAMPLE_DOCS):
    """
    Demonstrates RAG (Retrieval-Augmented Generation) functionality by analyzing sentiment
    of a collection of documents and using an LLM to provide an overall sentiment summary.
    
    Args:
        docs (list): A list of text documents to analyze. Defaults to SAMPLE_DOCS.
        
    Returns:
        None: Results are printed to standard output.
    """
    if not docs:
        print("[Theorem13 RAG Demo A] No docs.")
        return
    sentiments = [(d, sent_analyzer.analyze(d)) for d in docs]
    prompt = f"Docs with sentiment:\n{sentiments}\nProvide an overall sentiment analysis."
    resp = call_groq_llm(prompt)
    print("[Theorem13 RAG Demo A] LLM:\n", resp, "\n")

def rag_demo_b(user_query, docs=SAMPLE_DOCS):
    """
    Demonstrates RAG functionality by selecting a random document, analyzing its sentiment,
    and using an LLM to generate a response to the user query with sentiment context.
    
    Args:
        user_query (str): The query from the user to answer.
        docs (list): A list of text documents from which to select. Defaults to SAMPLE_DOCS.
        
    Returns:
        None: Results are printed to standard output.
    """
    if not docs:
        print("[Theorem13 RAG Demo B] No docs.")
        return
    chosen_doc = random.choice(docs)
    sentiment = sent_analyzer.analyze(chosen_doc)
    prompt = (
        f"User query: {user_query}\nChosen doc:\n{chosen_doc}\n"
        f"Sentiment: {sentiment}\nConstruct an answer referencing sentiment."
    )
    resp = call_groq_llm(prompt)
    print("[Theorem13 RAG Demo B] LLM:\n", resp, "\n")

In [ ]:
rag_demo_a(SAMPLE_DOCS)


[Theorem13 RAG Demo A] LLM:
 [Groq LLM] Based on the provided text data, here's an overall sentiment analysis:

1. The first document, "Neural Networks: An Overview", has a very positive overall sentiment with a compound score of 0.953. The text discusses the concept, types, and applications of neural networks, highlighting their potential and importance in artificial intelligence.

2. The second document, "Generative AI and Blockchain Integration", also has a very positive overall sentiment with a compound score of 0.8442. The text explores the integration of generative AI and blockchain, discussing potential applications and benefits.

3. The third document, "Quantum Computing", has a very positive overall sentiment with a compound score of 0.9335. The text introduces the concept of quantum computing, its potential impact on various fields, and the challenges it faces.

Overall, the sentiment analysis of the three documents shows a predominantly positive tone, indicating enthusiasm a

In [ ]:
rag_demo_b("Are people positive about AI?", SAMPLE_DOCS)


[Theorem13 RAG Demo B] LLM:
 [Groq LLM] The overall sentiment towards AI in the provided text is very positive, with a compound sentiment score of 0.8442. The text highlights the potential benefits and opportunities of integrating generative AI and blockchain technologies, such as creating unique digital assets, enabling decentralized AI training, and providing transparent and verifiable AI inference. While there are some challenges and considerations mentioned, such as computational limitations and privacy concerns, the overall tone of the text is optimistic and enthusiastic about the potential of AI. 

